In [1]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import tifffile as tf
import os
import gdown
from PIL import Image

/Users/aj/Desktop/work/virtualenvs/edx/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Download data

In [5]:
url_abundance_maps = "https://drive.google.com/uc?export=download&id=1Wwkd5HeddZowBTMnsSfDGfX6Z0cB3LSu"
gdown.download(url_abundance_maps,'../../../primary_data/abundance_maps.npz',quiet='False')

'../../../primary_data/abundance_maps.npz'

## Load data

In [2]:
abundance_maps = (np.load('../../../primary_data/abundance_maps.npz')['arr_0'])/255
print(abundance_maps.shape)

(11, 1024, 1024, 30)


In [3]:
# select the abundance maps to be considered
indices = [3,4,6,1,9,7]
colors = [[1,0,0], [0,1,0], [0,0,1], [0,1,1], [1,0,1], [1,1,0]]

abundance_maps_selected = np.zeros((len(indices), abundance_maps.shape[1], abundance_maps.shape[2], abundance_maps.shape[3]))

for i in range(len(indices)):
    abundance_maps_selected[i,:,:,:] = abundance_maps[indices[i],:,:,:]
print(np.max(abundance_maps_selected))

1.0


In [8]:
# pixel-wise coloring based on the most dominant endmember with intensity scaled its weight

rgb_image = np.zeros((3,abundance_maps_selected.shape[1], abundance_maps_selected.shape[2], abundance_maps_selected.shape[3]))

for t in range(rgb_image.shape[3]):
    for x in range(rgb_image.shape[1]):
        for y in range(rgb_image.shape[2]):
            idx = np.argmax(abundance_maps_selected[:,x,y,t])
            rgb_image[0,x,y,t] = colors[idx][0] * abundance_maps_selected[idx,x,y,t]
            rgb_image[1,x,y,t] = colors[idx][1] * abundance_maps_selected[idx,x,y,t]
            rgb_image[2,x,y,t] = colors[idx][2] * abundance_maps_selected[idx,x,y,t]
    rgb_tiff = (np.copy(rgb_image[:,:,:,t]).transpose(1,2,0)*255).astype('uint8')
    tf.imwrite('../../secondary_data/main_mosaic_5by6_multicolor_tiffs/' + 'Tile_' + '%02d' %(t) + '.tiff', rgb_tiff, photometric = 'rgb')
    im_png = Image.fromarray(rgb_tiff)
    im_png.save('../../../primary_data/main_mosaic_6by5/multicolor_png/' + 'Tile_' + '%02d' %(t) + '.png')
    

### Option: save subsequent (in abundance) maps (not in paper) 

In [4]:
# pixel-wise coloring based on the most dominant endmember with intensity scaled its weight

abundance_order = 2

rgb_image = np.zeros((3,abundance_maps_selected.shape[1], abundance_maps_selected.shape[2], abundance_maps_selected.shape[3]))

try:
    os.mkdir('../../../primary_data/main_mosaic_6by5/multicolor_png_abundance_order%02d/' % abundance_order)
except:
    print('dir exists')

for t in range(rgb_image.shape[3]):
    for x in range(rgb_image.shape[1]):
        for y in range(rgb_image.shape[2]):
            idx = np.argsort(abundance_maps_selected[:,x,y,t])[-abundance_order]
            rgb_image[0,x,y,t] = colors[idx][0] * abundance_maps_selected[idx,x,y,t]
            rgb_image[1,x,y,t] = colors[idx][1] * abundance_maps_selected[idx,x,y,t]
            rgb_image[2,x,y,t] = colors[idx][2] * abundance_maps_selected[idx,x,y,t]
    rgb_tiff = (np.copy(rgb_image[:,:,:,t]).transpose(1,2,0)*255).astype('uint8')
    #tf.imwrite('../../secondary_data/main_mosaic_5by6_multicolor_tiffs/' + 'Tile_' + '%02d' %(t) + '.tiff', rgb_tiff, photometric = 'rgb')
    im_png = Image.fromarray(rgb_tiff)
    im_png.save('../../../primary_data/main_mosaic_6by5/multicolor_png_abundance_order%02d/' % abundance_order + 'Tile_' + '%02d' %(t) + '.png')
    